In [1]:
import pandas as pd

In [2]:
csv_file_path = "match_results.csv"
df = pd.read_csv(csv_file_path)

In [3]:
artifact_count = df.groupby("vulnerable artifact")["Matching Artifact"].count()

artifact_counts_sorted = artifact_count.sort_values(ascending=False)

In [ ]:
# write this to a file
artifact_counts_sorted.to_csv("artifact_counts_sorted.csv")

print(artifact_counts_sorted)

In [ ]:
from pymongo import MongoClient


def connect_to_mongodb():
    client = MongoClient("mongodb://localhost:27072/")
    db = client.osv_db
    return db


mongo_db = connect_to_mongodb()


def is_vulnerable(library, version):
    query = {
        "affected.package.name": library,
        "affected.package.ecosystem": "Maven",
        "affected.versions": version,
    }
    return mongo_db.data.find(query)

In [ ]:
not_reported_vulnerabilities_count = 0
artifacts_not_reported = []
for index, row in df.iterrows():
    group_id, artifact_id, version = row["Matching Artifact"].split(":")
    library = f"{group_id}:{artifact_id}"
    # print(f"Checking {library} {version}")
    vulnerabilities = list(is_vulnerable(library, version))
    if len(vulnerabilities) == 0:
        not_reported_vulnerabilities_count += 1
        artifacts_not_reported.append((group_id, artifact_id, version))
        # print(f"Vulnerability not reported for {library} {version}")
    
print(f"Total not reported vulnerabilities: {not_reported_vulnerabilities_count}")


In [ ]:
# write this to a file
with open("artifacts_not_reported.txt", "w") as f:
    for artifact in artifacts_not_reported:
        f.write(f"{artifact[0]}:{artifact[1]}:{artifact[2]}\n")

In [ ]:
from mysql.connector import pooling
from jproperties import Properties

properties = Properties()
with open("../config.properties", "rb") as properties_file:
    properties.load(properties_file, "utf-8")


def parse_database_url(db_url):
    # db_url is in the format "jdbc:postgresql://localhost:5432/maven"
    try:
        url_parts = db_url.split("//")[1].split("/")
        host_port = url_parts[0]
        database = url_parts[1]

        host = host_port.split(":")[0]

        return host, database
    except IndexError:
        raise ValueError("Invalid database URL format")

db_host, db_name = parse_database_url(properties.get("database.url").data)


def connect_to_db():
    try:
        connection_pool = pooling.MySQLConnectionPool(
            pool_name="pom_resolution_pool",
            pool_size=5,
            host=db_host,
            database=db_name,
            user=properties.get("database.username").data,
            password=properties.get("database.password").data,
        )
        return connection_pool
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

In [ ]:
creation_dates = []
connection_pool = connect_to_db()
connection_object = connection_pool.get_connection()
if connection_pool is None:
    print("Error connecting to the database")
else:
    for artifact in artifacts_not_reported:



        cursor = connection_object.cursor()

        group_id, artifact_id, version = artifact
        query = f"SELECT creation_date FROM libraries_big WHERE group_id = '{group_id}' AND artifact_id = '{artifact_id}' AND version = '{version}'"
        cursor.execute(query)
        result = cursor.fetchall()
        # creation_date is of type datetime DEFAULT NULL in mariadb
        if len(result) != 0:
            creation_dates.append(result[0][0])

cursor.close()
connection_object.close()

In [ ]:
# write this to a file
with open("non_reported_artifacts_creation_dates.txt", "w") as f:
    for date in creation_dates:
        f.write(f"{date}\n")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

creation_dates = np.array(creation_dates)
years = mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')

plt.rcParams.update(
        {
            "font.family": "serif",
            "text.usetex": True,
            "pgf.rcfonts": False,
            "pgf.preamble": r"\usepackage{times}" + "\n" + r"\usepackage{mathptmx}",
        }
    )

plt.figure(figsize=(10, 5))
n, bins, patches = plt.hist(creation_dates, bins=80, color='skyblue', edgecolor='black')


plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(years_fmt)

plt.title('Distribution of vulnerable artifacts creation dates', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of vulnerable artifacts', fontsize=12)

plt.gcf().autofmt_xdate()

plt.grid(True)

plt.tight_layout()
plt.savefig('vulnerable_artifacts_creation_dates.pdf', format='pdf')

plt.show()